# 기상데이터 병합 및 일자별 평균

* 각 관측소마다 일시별로 측정된 데이터를 일시별로 평균냄

In [4]:
# 필요 라이브러리 import
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
from urllib import parse
import os

In [15]:
data = pd.read_excel('data/raw_data/미세먼지/2019년 7월.xlsx')
data.head(3)

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소
0,서울 중구,도시대기,111121,중구,2019070101,0.002,0.5,0.055,0.010,40.0,29.0,서울 중구 덕수궁길 15
1,서울 중구,도시대기,111121,중구,2019070102,0.002,0.5,0.056,0.012,36.0,29.0,서울 중구 덕수궁길 15
2,서울 중구,도시대기,111121,중구,2019070103,0.003,0.7,0.055,0.011,37.0,30.0,서울 중구 덕수궁길 15


In [18]:
data = data.loc[(data['망'] == '도시대기')].reset_index(drop=True)

In [6]:
data['측정날짜'] = data['측정일시'].astype('str').str[:-2]

data['측정일시'] = data['측정일시'].astype('str').str[-2:].replace('24', '00')

In [8]:
data['지역'] = data['지역'].str[:2]

In [12]:
data = data.loc[(data['지역'] == '서울') | (data['지역'] == '경기')].reset_index(drop=True)

In [19]:
data

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소


In [14]:
data.groupby(['지역', '측정날짜', '측정일시']).mean().iloc[:, 1:].reset_index()

,지역,측정날짜,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,경기,20190701,00,0.003209,0.494048,0.042465,0.014953,30.529412,20.559524
1,경기,20190701,01,0.002779,0.463095,0.052012,0.006400,49.541176,36.216867
2,경기,20190701,02,0.002907,0.490476,0.054435,0.005918,54.690476,38.964286
3,경기,20190701,03,0.003058,0.503571,0.054671,0.006024,52.011905,38.154762
4,경기,20190701,04,0.003070,0.522619,0.052812,0.006553,48.048193,36.590361
...,...,...,...,...,...,...,...,...,...
1483,서울,20190731,19,0.003600,0.443478,0.012160,0.029840,20.956522,11.391304
1484,서울,20190731,20,0.003440,0.439130,0.011120,0.027000,24.083333,13.458333
1485,서울,20190731,21,0.003400,0.365217,0.016583,0.018200,24.920000,14.320000
1486,서울,20190731,22,0.003440,0.343478,0.016542,0.017560,24.960000,14.086957


In [26]:
## 함수화


pollution = pd.DataFrame()
base_path = 'data/raw_data/미세먼지/'
file_list = os.listdir('data/raw_data/미세먼지/')

for file in tqdm(file_list[1:]):

    # 파일 경로 결합 및 불러오기
    file_path = base_path + file
    data = pd.read_excel(file_path)
    
    # 도시 대기만 사용
    data = data.loc[(data['망'] == '도시대기')].reset_index(drop=True)
    
    
    data['지역'] = data['지역'].str[:2]
    # 서울, 경기만 추출
    data = data.loc[(data['지역'] == '서울') | (data['지역'] == '경기')].reset_index(drop=True)
    
    data['측정날짜'] = data['측정일시'].astype('str').str[:-2]
    data['측정일시'] = data['측정일시'].astype('str').str[-2:].replace('24', '00')
    
    grouped_data = data.groupby(['지역', '측정날짜', '측정일시']).mean().reset_index()
    display(grouped_data)
#     grouped_data = data.groupby(['측정날짜', '행정구역']).mean().iloc[:, 1:].reset_index()
    
    pollution = pd.concat([pollution, grouped_data], axis=0)
    
    

,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20191001,00,163870.732558,0.003386,0.502439,0.007771,0.026566,44.475610,28.268293
1,경기,20191001,01,163870.732558,0.003183,0.676543,0.013753,0.038024,80.641975,52.650000
2,경기,20191001,02,163870.732558,0.003110,0.669136,0.012580,0.035512,83.543210,55.400000
3,경기,20191001,03,163870.732558,0.003159,0.661728,0.011506,0.033683,79.271605,53.925000
4,경기,20191001,04,163870.732558,0.003098,0.643210,0.010444,0.031854,75.135802,51.875000
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20191031,19,111211.040000,0.003840,0.624000,0.048760,0.028120,157.958333,40.791667
1484,서울,20191031,20,111211.040000,0.003720,0.640000,0.047440,0.029520,169.833333,45.560000
1485,서울,20191031,21,111211.040000,0.003720,0.672000,0.042520,0.031120,169.000000,46.320000
1486,서울,20191031,22,111211.040000,0.003600,0.656000,0.037240,0.032400,159.583333,44.240000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20191101,00,163870.732558,0.003560,0.697590,0.003976,0.044869,58.493976,29.142857
1,경기,20191101,01,163870.732558,0.003250,0.581707,0.030298,0.025119,152.280488,42.694118
2,경기,20191101,02,163870.732558,0.003179,0.600000,0.029214,0.022881,152.731707,44.564706
3,경기,20191101,03,163870.732558,0.003083,0.610976,0.027393,0.021381,148.439024,45.070588
4,경기,20191101,04,163870.732558,0.002833,0.618293,0.025155,0.020821,139.902439,43.623529
...,...,...,...,...,...,...,...,...,...,...
1435,서울,20191130,19,111211.040000,0.005160,0.684000,0.003520,0.055840,66.840000,42.916667
1436,서울,20191130,20,111211.040000,0.004840,0.664000,0.004400,0.051640,55.958333,35.791667
1437,서울,20191130,21,111211.040000,0.004880,0.696000,0.003600,0.053000,53.320000,35.347826
1438,서울,20191130,22,111211.040000,0.004880,0.684000,0.004080,0.053080,50.080000,33.250000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20191201,00,163131.715909,0.002845,0.504762,0.019541,0.019082,31.960000,26.600000
1,경기,20191201,01,163131.715909,0.004833,0.720238,0.003071,0.048059,60.195122,40.279070
2,경기,20191201,02,163131.715909,0.004512,0.705952,0.003294,0.045765,58.619048,40.267442
3,경기,20191201,03,163131.715909,0.004226,0.691667,0.003424,0.043718,55.566265,39.151163
4,경기,20191201,04,163131.715909,0.004155,0.675000,0.003424,0.042012,52.060241,37.069767
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20191231,19,111211.040000,0.003160,0.444000,0.015000,0.026320,22.200000,13.040000
1484,서울,20191231,20,111211.040000,0.003080,0.456000,0.014640,0.026280,24.240000,14.560000
1485,서울,20191231,21,111211.040000,0.003120,0.444000,0.015160,0.024680,24.640000,15.280000
1486,서울,20191231,22,111211.040000,0.002960,0.464000,0.013920,0.024920,22.800000,14.240000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20190701,00,163870.732558,0.003209,0.494048,0.042465,0.014953,30.529412,20.559524
1,경기,20190701,01,163870.732558,0.002779,0.463095,0.052012,0.006400,49.541176,36.216867
2,경기,20190701,02,163870.732558,0.002907,0.490476,0.054435,0.005918,54.690476,38.964286
3,경기,20190701,03,163870.732558,0.003058,0.503571,0.054671,0.006024,52.011905,38.154762
4,경기,20190701,04,163870.732558,0.003070,0.522619,0.052812,0.006553,48.048193,36.590361
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20190731,19,111211.040000,0.003600,0.443478,0.012160,0.029840,20.956522,11.391304
1484,서울,20190731,20,111211.040000,0.003440,0.439130,0.011120,0.027000,24.083333,13.458333
1485,서울,20190731,21,111211.040000,0.003400,0.365217,0.016583,0.018200,24.920000,14.320000
1486,서울,20190731,22,111211.040000,0.003440,0.343478,0.016542,0.017560,24.960000,14.086957


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20190801,00,163870.732558,0.002805,0.315000,0.008370,0.012220,18.756410,9.564103
1,경기,20190801,01,163870.732558,0.002702,0.322222,0.020580,0.010179,19.370370,8.250000
2,경기,20190801,02,163870.732558,0.002619,0.324691,0.020383,0.009190,19.395062,8.012987
3,경기,20190801,03,163870.732558,0.002643,0.311111,0.020568,0.007619,15.974026,8.402597
4,경기,20190801,04,163870.732558,0.002571,0.307407,0.019000,0.007155,12.443038,5.961039
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20190831,19,111211.040000,0.003625,0.408696,0.035875,0.019875,18.909091,10.909091
1484,서울,20190831,20,111211.040000,0.003875,0.434783,0.031542,0.021417,20.826087,11.285714
1485,서울,20190831,21,111211.040000,0.003750,0.439130,0.029292,0.020750,19.391304,11.227273
1486,서울,20190831,22,111211.040000,0.003583,0.434783,0.026542,0.021083,18.136364,10.681818


...

,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20200501,00,159615.474747,0.002735,0.430928,0.045816,0.008454,54.422680,31.091837
1,경기,20200501,01,159615.474747,0.002773,0.402083,0.042680,0.008458,42.052083,21.851064
2,경기,20200501,02,159615.474747,0.002722,0.395833,0.042216,0.007792,42.645833,23.191489
3,경기,20200501,03,159615.474747,0.002619,0.386458,0.042134,0.007354,41.907216,23.648936
4,경기,20200501,04,159615.474747,0.002619,0.386458,0.040052,0.007646,40.917526,24.382979
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20200531,19,111211.040000,0.005240,0.400000,0.056600,0.014160,37.560000,27.800000
1484,서울,20200531,20,111211.040000,0.004480,0.382609,0.053280,0.012800,35.640000,27.000000
1485,서울,20200531,21,111211.040000,0.003560,0.382609,0.052800,0.012040,33.291667,25.400000
1486,서울,20200531,22,111211.040000,0.003040,0.382609,0.052880,0.011240,33.320000,25.320000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20200601,00,159615.474747,0.002979,0.372165,0.036577,0.015812,22.833333,13.340206
1,경기,20200601,01,159615.474747,0.002598,0.371429,0.050020,0.006175,22.813187,15.780220
2,경기,20200601,02,159615.474747,0.002526,0.350000,0.050541,0.004918,18.336842,14.228261
3,경기,20200601,03,159615.474747,0.002495,0.337755,0.050949,0.004454,16.391304,13.021277
4,경기,20200601,04,159615.474747,0.002485,0.333673,0.050653,0.004340,15.231579,11.562500
...,...,...,...,...,...,...,...,...,...,...
1435,서울,20200630,19,111211.040000,0.002240,0.256000,0.037120,0.010320,6.440000,2.583333
1436,서울,20200630,20,111211.040000,0.002160,0.252000,0.034360,0.009520,6.200000,2.208333
1437,서울,20200630,21,111211.040000,0.002160,0.244000,0.033120,0.008320,6.320000,3.083333
1438,서울,20200630,22,111211.040000,0.002200,0.268000,0.029200,0.011120,5.875000,2.750000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20200701,00,159059.871287,0.002742,0.471277,0.020957,0.018505,46.591398,27.414894
1,경기,20200701,01,159059.871287,0.002463,0.316842,0.023284,0.011074,10.163043,4.720430
2,경기,20200701,02,159059.871287,0.002547,0.324211,0.022874,0.010064,10.223404,4.957447
3,경기,20200701,03,159059.871287,0.002621,0.317895,0.022737,0.008660,10.810526,4.709677
4,경기,20200701,04,159059.871287,0.002547,0.312632,0.021326,0.008138,11.600000,4.787234
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20200731,19,111211.040000,0.003400,0.328000,0.011320,0.020320,16.320000,9.800000
1484,서울,20200731,20,111211.040000,0.003160,0.348000,0.010400,0.021120,17.200000,9.880000
1485,서울,20200731,21,111211.040000,0.003040,0.332000,0.008480,0.021480,19.875000,11.360000
1486,서울,20200731,22,111211.040000,0.002960,0.328000,0.006560,0.020240,19.541667,12.240000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20200801,00,159059.871287,0.002531,0.296939,0.022701,0.007459,12.835052,5.020408
1,경기,20200801,01,159059.871287,0.002566,0.295960,0.009184,0.008556,15.824742,6.867347
2,경기,20200801,02,159059.871287,0.002576,0.287879,0.009418,0.007747,15.051020,6.040404
3,경기,20200801,03,159059.871287,0.002636,0.288889,0.008949,0.007182,14.422680,5.585859
4,경기,20200801,04,159059.871287,0.002646,0.297980,0.007469,0.007576,15.275510,5.979798
...,...,...,...,...,...,...,...,...,...,...
1483,서울,20200831,19,111211.040000,0.002400,0.350000,0.017320,0.018640,11.320000,5.200000
1484,서울,20200831,20,111211.040000,0.002400,0.337500,0.014720,0.018000,10.440000,4.708333
1485,서울,20200831,21,111211.040000,0.002560,0.320833,0.015080,0.015280,9.640000,4.333333
1486,서울,20200831,22,111211.040000,0.002560,0.312500,0.014800,0.014440,8.080000,3.400000


,지역,측정날짜,측정일시,측정소코드,SO2,CO,O3,NO2,PM10,PM25
0,경기,20200901,00,158790.088235,0.002745,0.424490,0.017837,0.017667,27.229167,12.858586
1,경기,20200901,01,158790.088235,0.002670,0.332292,0.019439,0.005286,8.711340,3.263158
2,경기,20200901,02,158790.088235,0.002608,0.325000,0.019918,0.004684,9.195876,2.860215
3,경기,20200901,03,158790.088235,0.002680,0.316667,0.019969,0.004531,9.917526,3.093750
4,경기,20200901,04,158790.088235,0.002629,0.309375,0.020439,0.004429,10.350515,3.319149
...,...,...,...,...,...,...,...,...,...,...
1435,서울,20200930,19,111211.040000,0.002600,0.428000,0.034320,0.018208,31.320000,21.120000
1436,서울,20200930,20,111211.040000,0.002520,0.420000,0.032160,0.016458,27.520000,20.040000
1437,서울,20200930,21,111211.040000,0.002560,0.444000,0.023080,0.017625,22.720000,16.680000
1438,서울,20200930,22,111211.040000,0.002360,0.424000,0.017040,0.019167,19.600000,14.320000


In [33]:
pollution = pollution.sort_values(by=['지역', '측정날짜', '측정일시'], ascending=True).reset_index(drop=True)

In [40]:
pollution_final = pollution.loc[pollution['측정날짜'] >='20190717'].reset_index(drop=True)

pollution_final.drop('측정소코드', axis=1, inplace=True)

In [43]:
pollution_final = np.round(pollution_final, 3)

In [46]:
pollution_final.to_csv('data/가공/지역별미세먼지데이터_최종.csv', encoding='utf-8')

In [47]:
pollution_final

,지역,측정날짜,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,경기,20190717,00,0.004,0.628,0.025,0.027,79.512,54.272
1,경기,20190717,01,0.003,0.543,0.013,0.023,57.704,39.325
2,경기,20190717,02,0.003,0.538,0.011,0.022,61.753,39.952
3,경기,20190717,03,0.003,0.533,0.010,0.020,59.707,39.904
4,경기,20190717,04,0.003,0.530,0.009,0.019,56.691,37.790
...,...,...,...,...,...,...,...,...,...
21211,서울,20200930,19,0.003,0.428,0.034,0.018,31.320,21.120
21212,서울,20200930,20,0.003,0.420,0.032,0.016,27.520,20.040
21213,서울,20200930,21,0.003,0.444,0.023,0.018,22.720,16.680
21214,서울,20200930,22,0.002,0.424,0.017,0.019,19.600,14.320
